# Snowflake Data Analytics Notebook
This notebook demonstrates how to connect to Snowflake and perform data analytics operations with SSO authentication.


## 1. Setup and Imports

**Run this cell first!**


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from snowflake_connector import SnowflakeConnector

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Imports successful!")


✅ Imports successful!


## 2. Connect to Snowflake (SSO)

**IMPORTANT:** Run this cell and complete the browser SSO login before proceeding!


In [2]:
# Initialize connection (uses .env file configuration)
# For SSO: Make sure SNOWFLAKE_AUTHENTICATOR=externalbrowser in your .env file
sf = SnowflakeConnector()

# Connect to Snowflake - browser will open for SSO authentication
print("⏳ Connecting to Snowflake...")
print("🌐 A browser window will open for SSO login...")
print("   Please complete the authentication in your browser.")
print()

if sf.connect():
    print("✅ Successfully connected to Snowflake!")
    print("   You can now run queries!")
else:
    print("❌ Failed to connect to Snowflake.")
    print("   Please check your .env file and credentials.")
    print("   See SETUP_FOR_SSO.md for help.")


INFO:snowflake_connector:Using SSO authentication (external browser)
INFO:snowflake_connector:Opening browser for SSO authentication...
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.12.2, Python Version: 3.13.3, Platform: macOS-26.0.1-arm64-arm-64bit-Mach-O
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


⏳ Connecting to Snowflake...
🌐 A browser window will open for SSO login...
   Please complete the authentication in your browser.

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://doordash.okta.com/app/snowflake/exkkgmn09wbgWonqa0x7/sso/saml?SAMLRequest=jZJBb%2BIwEIX%2FSuQ9EzspFa0FVClZ1GwpRBBaaW8mMeAmsYPHIXR%2F%2FTqhrLqHVr1Z4%2Ffsb%2BbN8O5UFs6RaxBKjpDnEuRwmapMyN0IrZNp7wY5YJjMWKEkH6E3DuhuPARWFhUNarOXS36oORjHPiSBdhcjVGtJFQMBVLKSAzUpXQVPM%2Bq7hFZaGZWqAn2wfO1gAFwbS3ixZCAs3t6YimLcNI3bXLlK77BPCMHkFltVK%2Flx0Z9sT5%2FoPUz6rd4qrDx%2BZ7sX8jyCr7A2ZxHQhySJe%2FFilSAnuKBOlIS65HrF9VGkfL2cnQHAEoSLxTIMVg8uSNVsC5bzVJVVbexjrj3hLc9woXbC9huFI1TlInvKD%2F2Qsc1kOtk%2FPv46bqM%2Fm9cynt9fVa%2FrJHjhx4GOf86iwZykyHm%2BBOq3gUYANY9kG6OxJeJf9zzS864Tn1Di0f6NS3zvN3JCG6OQzHTOC2umlM4Y7F2VG9bRsarC%2F8AxP%2BX5rpTkttnsXp

INFO:snowflake_connector:Successfully connected to Snowflake


✅ Successfully connected to Snowflake!
   You can now run queries!


## 3. Verify Connection

Check that you're connected and see your current Snowflake context.


In [3]:
# Verify connection and show current context
query = """
SELECT 
    CURRENT_TIMESTAMP() as current_time,
    CURRENT_USER() as user,
    CURRENT_DATABASE() as database,
    CURRENT_SCHEMA() as schema,
    CURRENT_WAREHOUSE() as warehouse,
    CURRENT_ROLE() as role
"""
df_info = sf.query_to_dataframe(query)
print("📊 Your Snowflake Connection Info:")
print("=" * 60)
for col in df_info.columns:
    print(f"{col.upper()}: {df_info[col].iloc[0]}")
print("=" * 60)


INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake_connector:Query returned DataFrame with shape (1, 6)


📊 Your Snowflake Connection Info:
CURRENT_TIME: 2025-10-15 20:01:59.724000+00:00
USER: YISHU.GU
DATABASE: PRODDB
SCHEMA: PUBLIC
WAREHOUSE: ADHOC
ROLE: YISHUGU


## 4. Query Your Data

Now you can run queries! Here's your FACT_DEDUP_EXPERIMENT_EXPOSURE table query.


In [4]:
# Query from FACT_DEDUP_EXPERIMENT_EXPOSURE table
query = """
SELECT * 
FROM PRODDB.PUBLIC.FACT_DEDUP_EXPERIMENT_EXPOSURE 
LIMIT 10
"""

print("🔍 Running query...")
df = sf.query_to_dataframe(query)
print(f"✅ Retrieved {len(df)} rows with {len(df.columns)} columns")
print()
df.head()


🔍 Running query...


INFO:snowflake.connector.cursor:Number of results in first chunk: 10
INFO:snowflake_connector:Query returned DataFrame with shape (10, 17)


✅ Retrieved 10 rows with 17 columns



,EXPERIMENT_NAME,EXPERIMENT_VERSION,BUCKET_KEY,RESULT,TAG,SEGMENT,CUSTOM_DISTRIBUTION,EXPOSURE_TIME,EVENT_SENT_AT,DEDUPLICATED_AT,EVENT_COUNT,BUCKET_KEY_TYPE,CALLING_CONTEXT,CUSTOM_ATTRIBUTES,RECEIVED_AT,RESOLUTION_LOG,SERVICE
0,enable_reorder_info_in_preview_response,1,94501336,true,treatment,unset_segment,None,2023-01-16 02:46:17.916,2023-01-16 02:46:17.917,2023-01-17 08:02:56.073,8,None,None,None,None,None,None
1,enable_nv_flat_fee_exp_v2,5,157716490,control,control,unset_segment,None,2023-01-16 22:51:10.425,2023-01-16 22:51:10.426,2023-01-17 08:02:56.073,9,None,None,None,None,None,None
2,enable_retail_hsa_fsa_item_tagging,1,1202394201,treatment,treatment,All Users,Equal Weights,2023-01-16 03:37:16.967,2023-01-16 03:37:16.967,2023-01-17 08:02:56.073,1,None,None,None,None,None,None
3,enable_gift_tracking_iterable_campaigns,1,886289438,true,treatment,unset_segment,None,2023-01-16 17:26:41.861,2023-01-16 17:26:41.862,2023-01-17 08:02:56.073,2,None,None,None,None,None,None
4,enable_log_for_effective_tax_category_compare,1,1673840562160,true,true,unset_segment,None,2023-01-16 03:42:42.160,2023-01-16 03:42:42.227,2023-01-17 08:02:56.073,1,None,None,None,None,None,None


## 5. Explore Your Data

Let's look at the data structure and basic statistics.


In [ ]:
# Show column names and types
print("📋 Column Information:")
print("=" * 60)
for col in df.columns:
    print(f"{col}: {df[col].dtype}")
print("=" * 60)
print(f"\nDataFrame shape: {df.shape}")


In [ ]:
# Basic statistics for numeric columns
df.describe()


In [ ]:
# Check for missing values
print("Missing values per column:")
df.isnull().sum()


## 6. Custom Queries

Add your own SQL queries here!


In [ ]:
# Example: Your custom query
# Uncomment and modify as needed
# custom_query = """
# SELECT 
#     column1,
#     COUNT(*) as count
# FROM PRODDB.PUBLIC.FACT_DEDUP_EXPERIMENT_EXPOSURE
# GROUP BY column1
# ORDER BY count DESC
# LIMIT 20
# """
# df_custom = sf.query_to_dataframe(custom_query)
# df_custom


## 7. Data Visualization

Create visualizations of your data.


In [ ]:
# Example: Bar chart (uncomment and modify column name)
# plt.figure(figsize=(12, 6))
# df['your_column'].value_counts().head(10).plot(kind='bar')
# plt.title('Top 10 Values')
# plt.xlabel('Category')
# plt.ylabel('Count')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()


## 8. Export Results

Save your query results to files.


In [ ]:
# Export to CSV
# df.to_csv('experiment_exposure_data.csv', index=False)
# print("✅ Data exported to experiment_exposure_data.csv")


## 9. Close Connection

When you're done, close the Snowflake connection.


In [ ]:
# Close the Snowflake connection
sf.close()
print("✅ Connection closed")


# Snowflake Data Analytics Notebook
This notebook demonstrates how to connect to Snowflake and perform data analytics operations.


## 1. Setup and Imports


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from snowflake_connector import SnowflakeConnector

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Imports successful!")


## 2. Connect to Snowflake


In [ ]:
# Initialize connection (uses .env file configuration)
# For SSO: Make sure SNOWFLAKE_AUTHENTICATOR=externalbrowser in your .env file
# A browser window will open for SSO authentication
sf = SnowflakeConnector()

# Connect to Snowflake
print("⏳ Connecting to Snowflake...")
if sf.connect():
    print("✅ Successfully connected to Snowflake!")
    print("(If using SSO, you should have completed login in your browser)")
else:
    print("❌ Failed to connect to Snowflake. Please check your credentials.")


## 3. Explore Available Tables


In [ ]:
# List all available tables
tables = sf.list_tables()
print(f"Found {len(tables)} tables:")
tables


## 4. Query Data from Snowflake


In [ ]:
# Get current connection info
query = """
SELECT 
    CURRENT_TIMESTAMP() as current_time,
    CURRENT_USER() as user,
    CURRENT_DATABASE() as database,
    CURRENT_SCHEMA() as schema,
    CURRENT_WAREHOUSE() as warehouse,
    CURRENT_ROLE() as role
"""
df_info = sf.query_to_dataframe(query)
df_info


In [4]:
# Example: Query from FACT_DEDUP_EXPERIMENT_EXPOSURE table
query = """
SELECT * 
FROM PRODDB.PUBLIC.FACT_DEDUP_EXPERIMENT_EXPOSURE 
LIMIT 10
"""

df = sf.query_to_dataframe(query)
print(f"Retrieved {len(df)} rows with {len(df.columns)} columns")
df.head()


ERROR:snowflake_connector:Failed to create DataFrame from query: 'NoneType' object has no attribute 'execute'


AttributeError: 'NoneType' object has no attribute 'execute'

## 5. Data Exploration and Analysis


In [ ]:
# Basic statistics
# df.describe()


In [ ]:
# Check for missing values
# df.isnull().sum()


In [ ]:
# Data types
# df.dtypes


## 6. Data Visualization Examples


In [ ]:
# Example: Bar chart
# plt.figure(figsize=(12, 6))
# df['column_name'].value_counts().head(10).plot(kind='bar')
# plt.title('Top 10 Categories')
# plt.xlabel('Category')
# plt.ylabel('Count')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()


In [ ]:
# Example: Line chart with Plotly
# fig = px.line(df, x='date_column', y='value_column', title='Trend Over Time')
# fig.show()


In [ ]:
# Example: Correlation heatmap
# numeric_cols = df.select_dtypes(include=[np.number]).columns
# correlation_matrix = df[numeric_cols].corr()
# 
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
# plt.title('Correlation Matrix')
# plt.tight_layout()
# plt.show()


## 7. Advanced Analytics Examples


In [ ]:
# Example: Aggregation query
# query = """
# SELECT 
#     category,
#     COUNT(*) as count,
#     AVG(value) as avg_value,
#     SUM(amount) as total_amount
# FROM your_table
# GROUP BY category
# ORDER BY count DESC
# """
# df_agg = sf.query_to_dataframe(query)
# df_agg


In [ ]:
# Example: Time-based analysis
# query = """
# SELECT 
#     DATE_TRUNC('day', timestamp_column) as date,
#     COUNT(*) as daily_count,
#     SUM(amount) as daily_total
# FROM your_table
# WHERE timestamp_column >= DATEADD(day, -30, CURRENT_DATE())
# GROUP BY date
# ORDER BY date
# """
# df_time = sf.query_to_dataframe(query)
# df_time


## 8. Export Results


en

In [ ]:
# Export to CSV
# df.to_csv('output_data.csv', index=False)
# print("✅ Data exported to output_data.csv")


In [ ]:
# Export to Excel
# df.to_excel('output_data.xlsx', index=False)
# print("✅ Data exported to output_data.xlsx")


## 9. Close Connection


In [ ]:
# Close the Snowflake connection
sf.close()
print("✅ Connection closed")
